In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 4.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=32ee7713d3a93d5781ae12b23c6c51f29e6e4a0c31d2b5c8f4e56581c427575f
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=2f92322d1e9403180b0cde81bd6b841beeead68a08a2d1ba2efe134a4b5ab698
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                              activation='relu',
                              input_shape=(28,28,1)
                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                              activation='relu'
                              
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(10,activation='softmax')

  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import kerastuner as kt
from kerastuner import RandomSearch

from kerastuner.engine.hyperparameters import HyperParameters


In [ ]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',
                          max_trials=5,directory='output',project_name='Mnist_Fashion')

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 26m 22s]
val_accuracy: 0.9038333296775818

Best val_accuracy So Far: 0.9089999794960022
Total elapsed time: 01h 33m 56s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        134448    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                232330    
Total params: 367,898
Trainable params: 367,898
Non-trainable params: 0
_________________________________________________________________
